In [2]:
import os
import random
import shutil
import itertools
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import albumentations as A
from tqdm.auto import tqdm

C:\Users\perce\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
BASE_DIR = 'learning'
FONT_DIR = os.path.join(BASE_DIR, 'fonts')
DATA_DIR = os.path.join(BASE_DIR, 'data')
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')

# Параметри генерації
IMAGE_SIZE = (64, 64)
CLASSES = list(range(10))
NEGATIVE_CLASS_ID = 10
FONT_SIZES = [38, 42, 46, 50, 54] # 5 варіантів розмірів
TRAIN_TEST_SPLIT = 0.8  # 80% на навчання (200 з 250)

# Параметри аугментації
# 4 типи, 6 рівнів (0 = вимкнено, 5 = макс)
AUG_LEVELS = list(range(6)) # 0, 1, 2, 3, 4, 5

# Кількість "сміття" для негативного класу (навчання)
# Зробимо ~10% від обсягу позитивного тренувального сету
# (200 * 6^4) * 0.1 ≈ 25,920. Округлимо до 25,000.
NEGATIVE_TRAIN_SAMPLES = 25000
NEGATIVE_TEST_SAMPLES = 1000 # Для тестування

In [6]:
if os.path.exists(DATA_DIR):
    print(f"Очищую стару директорію: {DATA_DIR}...")
    shutil.rmtree(DATA_DIR)

# Створення нової структури (включаючи клас "10")
for c in CLASSES + [NEGATIVE_CLASS_ID]:
    os.makedirs(os.path.join(TRAIN_DIR, str(c)), exist_ok=True)
    os.makedirs(os.path.join(TEST_DIR, str(c)), exist_ok=True)

print(f"Структуру директорій створено в {DATA_DIR}")

Структуру директорій створено в learning\data


In [5]:
print("--- Етап 2: Налаштування генераторів ---")

def create_base_image(digit_char, font_path, font_size, image_size=(64, 64)):
    """
    Створює чисте зображення з фоновим шумом та тремтінням позиції.
    """
    # 1. Створюємо фон з легким шумом
    bg_noise = np.random.randint(0, 40, image_size, dtype=np.uint8)
    image = Image.fromarray(bg_noise, 'L') # 'L' = 8-bit grayscale
    draw = ImageDraw.Draw(image)

    # 2. Завантажуємо шрифт
    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        font = ImageFont.load_default()

    # 3. Розраховуємо позицію (x, y) для центрування
    text_width, text_height = draw.textbbox((0,0), digit_char, font=font)[2:4]
    x = (image_size[0] - text_width) / 2
    y = (image_size[1] - text_height) / 2

    # 4. Додаємо "тремтіння" позиції
    x += random.randint(-4, 4)
    y += random.randint(-4, 4)

    # Малюємо білий текст (або ледь сірий)
    draw.text((x, y), digit_char, font=font, fill=random.randint(220, 255))

    # Конвертуємо в numpy для albumentations
    return np.array(image)


def get_noise_aug(level):
    if level == 0: return A.NoOp()
    var_limit = [5, 10, 20, 35, 50][level-1]
    return A.GaussNoise(var_limit=(var_limit, var_limit+10), p=1.0)

def get_stripes_aug(level):
    if level == 0: return A.NoOp()
    blur_limit = [3, 5, 7, 9, 12][level-1]
    return A.MotionBlur(blur_limit=(blur_limit, blur_limit+2), p=1.0)

def get_lighting_aug(level):
    if level == 0: return A.NoOp()
    limit = [0.1, 0.2, 0.3, 0.4, 0.5][level-1]
    return A.RandomBrightnessContrast(brightness_limit=limit, contrast_limit=limit, p=1.0)

def get_perspective_aug(level):
    if level == 0: return A.NoOp()
    scale = [0.02, 0.04, 0.06, 0.08, 0.1][level-1]
    rotate = [5, 10, 15, 20, 25][level-1]
    return A.ShiftScaleRotate(shift_limit=0.05, scale_limit=scale, rotate_limit=rotate, p=1.0)

def create_garbage_image(image_size=(64, 64)):
    """
    Генерує одне зображення "сміття".
    """
    # Вибираємо тип сміття
    garbage_type = random.choice(['noise', 'lines', 'letters', 'extreme_aug'])

    # 1. Просто фон
    image_np = np.random.randint(0, 50, image_size, dtype=np.uint8)
    image = Image.fromarray(image_np, 'L')
    draw = ImageDraw.Draw(image)

    if garbage_type == 'lines':
        # Малюємо 1-3 випадкові лінії
        for _ in range(random.randint(1, 3)):
            x1, y1 = random.randint(0, 64), random.randint(0, 64)
            x2, y2 = random.randint(0, 64), random.randint(0, 64)
            draw.line((x1, y1, x2, y2), fill=random.randint(100, 255), width=random.randint(1, 3))
        image_np = np.array(image)

    elif garbage_type == 'letters':
        # Малюємо випадкову літеру
        try:
            font_path = random.choice(font_files)
            font_size = random.choice(FONT_SIZES)
            font = ImageFont.truetype(font_path, font_size)
            char = random.choice("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")
            draw.text((5, 5), char, font=font, fill=random.randint(150, 255))
        except Exception:
            pass # Якщо шрифти не завантажились, просто пропустимо
        image_np = np.array(image)

    elif garbage_type == 'extreme_aug':
        # Беремо випадкову аугментацію і застосовуємо її до шуму
        aug_list = [
            A.GridDistortion(p=1.0),
            A.Blur(blur_limit=15, p=1.0),
            A.MotionBlur(blur_limit=20, p=1.0)
        ]
        aug = random.choice(aug_list)
        image_np = aug(image=image_np)['image']

    return image_np

print("Пайплайни аугментацій налаштовано.")

--- Етап 2: Налаштування генераторів ---
Пайплайни аугментацій налаштовано.


In [8]:
try:
    font_files = [os.path.join(FONT_DIR, f) for f in os.listdir(FONT_DIR) if f.endswith('.ttf')]
    if len(font_files) < 5:
        print(f"ПОПЕРЕДЖЕННЯ: Знайдено лише {len(font_files)} шрифтів у {FONT_DIR}. Бажано 5.")
        if not font_files:
            raise FileNotFoundError
except FileNotFoundError:
    print(f"ПОМИЛКА: Директорія {FONT_DIR} порожня або не знайдена. Зупинка.")
    # Тут можна зупинити блокнот або вийти
    raise

# 1. Створюємо "базовий" набір завдань (250 шт)
base_tasks = []
for digit in CLASSES:
    for font_path in font_files:
        for font_size in FONT_SIZES:
            base_tasks.append({
                "digit": digit,
                "font_path": font_path,
                "font_size": font_size,
                "id": f"{digit}_font_{os.path.basename(font_path).split('.')[0]}_size_{font_size}"
            })

random.shuffle(base_tasks)
split_index = int(len(base_tasks) * TRAIN_TEST_SPLIT)
train_tasks = base_tasks[:split_index]
test_tasks = base_tasks[split_index:]

print(f"Загалом {len(base_tasks)} базових зображень (10 цифр * {len(font_files)} шрифтів * {len(FONT_SIZES)} розмірів).")
print(f"Розділено на {len(train_tasks)} для train і {len(test_tasks)} для test.")

Загалом 250 базових зображень (10 цифр * 5 шрифтів * 5 розмірів).
Розділено на 200 для train і 50 для test.


In [12]:
print(f"\n--- Етап 4: Генерація TRAIN сету (Позитивні класи) ---")

# Створюємо всі 6^4 = 1296 комбінацій рівнів
aug_combinations = list(itertools.product(AUG_LEVELS, repeat=4))
print(f"Буде згенеровано {len(train_tasks)} * {len(aug_combinations)} = {len(train_tasks) * len(aug_combinations)} зображень.")

for task in tqdm(train_tasks, desc="Генерація TRAIN (0-9)"):
    digit = task['digit']
    base_id = task['id']

    # Генеруємо базове зображення ОДИН РАЗ
    base_img_np = create_base_image(str(digit), task['font_path'], task['font_size'])

    # Проходимо по ВСІМ 1296 комбінаціям
    for levels in aug_combinations:
        noise_level, stripes_level, light_level, persp_level = levels

        # Створюємо пайплайн для цієї конкретної комбінації
        pipeline = A.Compose([
            get_noise_aug(noise_level),
            get_stripes_aug(stripes_level),
            get_lighting_aug(light_level),
            get_perspective_aug(persp_level),
        ])

        # Застосовуємо
        augmented_img_np = pipeline(image=base_img_np)['image']

        # Зберігаємо
        aug_id = f"n{noise_level}_s{stripes_level}_l{light_level}_p{persp_level}"
        save_path = os.path.join(TRAIN_DIR, str(digit), f"{base_id}_{aug_id}.png")
        Image.fromarray(augmented_img_np).save(save_path)

print("Генерацію позитивного TRAIN сету завершено.")


--- Етап 4: Генерація TRAIN сету (Позитивні класи) ---
Буде згенеровано 200 * 1296 = 259200 зображень.


Генерація TRAIN (0-9):   0%|          | 0/200 [00:00<?, ?it/s]C:\Users\perce\miniconda3\Lib\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
C:\Users\perce\miniconda3\Lib\site-packages\albumentations\augmentations\blur\functional.py:232: UserWarning: blur_limit: Non-zero kernel sizes must be odd. Range (12, 14) automatically adjusted to (13, 15).
  result = _ensure_odd_values(result, info.field_name)
C:\Users\perce\AppData\Local\Temp\ipykernel_9184\291219708.py:37: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  return A.GaussNoise(var_limit=(var_limit, var_limit+10), p=1.0)
Генерація TRAIN (0-9): 100%|██████████| 200/200 [38:22<00:00, 11.51s/it]

Генерацію позитивного TRAIN сету завершено.


In [15]:
print(f"\n--- Етап 5: Генерація TRAIN сету (Негативний клас {NEGATIVE_CLASS_ID}) ---")
for i in tqdm(range(NEGATIVE_TRAIN_SAMPLES), desc="Генерація 'сміття' (Train)"):
    garbage_img_np = create_garbage_image(IMAGE_SIZE)
    save_path = os.path.join(TRAIN_DIR, str(NEGATIVE_CLASS_ID), f"garbage_{i:05d}.png")
    Image.fromarray(garbage_img_np).save(save_path)

print(f"Згенеровано {NEGATIVE_TRAIN_SAMPLES} 'сміттєвих' зображень для TRAIN.")


--- Етап 5: Генерація TRAIN сету (Негативний клас 10) ---


Генерація 'сміття' (Train):   0%|          | 0/25000 [00:00<?, ?it/s]C:\Users\perce\miniconda3\Lib\site-packages\albumentations\augmentations\blur\functional.py:232: UserWarning: blur_limit: Non-zero kernel sizes must be odd. Range (3, 20) automatically adjusted to (3, 21).
  result = _ensure_odd_values(result, info.field_name)
Генерація 'сміття' (Train): 100%|██████████| 25000/25000 [01:20<00:00, 308.65it/s]

Згенеровано 25000 'сміттєвих' зображень для TRAIN.


In [16]:
print(f"Генерую {len(test_tasks)} 'чистих' тестових зображень...")

for task in tqdm(test_tasks, desc="Генерація TEST (0-9)"):
    digit = task['digit']
    base_id = task['id']

    # Генеруємо ТІЛЬКИ базове зображення (з шумом фону і тремтінням)
    base_img_np = create_base_image(str(digit), task['font_path'], task['font_size'])

    # Зберігаємо
    save_path = os.path.join(TEST_DIR, str(digit), f"{base_id}_clean.png")
    Image.fromarray(base_img_np).save(save_path)

print("Генерацію позитивного TEST сету завершено.")

# 6. Генеруємо TEST сет (негативний клас 10)
print(f"\n--- Етап 7: Генерація TEST сету (Негативний клас {NEGATIVE_CLASS_ID}) ---")
for i in tqdm(range(NEGATIVE_TEST_SAMPLES), desc="Генерація 'сміття' (Test)"):
    garbage_img_np = create_garbage_image(IMAGE_SIZE)
    save_path = os.path.join(TEST_DIR, str(NEGATIVE_CLASS_ID), f"garbage_{i:05d}.png")
    Image.fromarray(garbage_img_np).save(save_path)

print(f"Згенеровано {NEGATIVE_TEST_SAMPLES} 'сміттєвих' зображень для TEST.")
print("\n--- ✅ УСЮ ГЕНЕРАЦІЮ ЗАВЕРШЕНО! ---")

Генерую 50 'чистих' тестових зображень...


Генерація TEST (0-9): 100%|██████████| 50/50 [00:02<00:00, 19.19it/s]


Генерацію позитивного TEST сету завершено.

--- Етап 7: Генерація TEST сету (Негативний клас 10) ---


Генерація 'сміття' (Test): 100%|██████████| 1000/1000 [00:11<00:00, 90.64it/s]

Згенеровано 1000 'сміттєвих' зображень для TEST.

--- ✅ УСЮ ГЕНЕРАЦІЮ ЗАВЕРШЕНО! ---
